## Real use cases for chat-tuned models

After the previous exercise, you are now capable of using the Poro-34B-chat model through the API that the `aitta_client` provides.

After the client confugurations we will create a function to get response that takes a promt as an input. It uses always the same model and the role is user. Only the content varies. 

After configuring the client, we will create a function to get responses that take a prompt as input. You can also customize the maximum token count. It always uses the same model, and the role is "user." Only the content varies. 

In this section, we will explore several real-world use cases where chat-tuned models can be applied. The following table of contents outlines the different use cases covered in this notebook.

# Table of content
1. [Sentiment analysis of feedback](#Sentiment-analysis-of-feedback)
2. [Text summarization](#Text-summarization)
3. [Retrieving relevant research papers](#Retrieving-relevant-research-papers)
4. [Named entity recognition (NER)](#NER)
5. [Extracting key topics](#Extracting-key-topics)
6. [Refining academic writing](#Refining-academic-writing)
7. [Translation from Finnish to English](#Translation)

In [ ]:
api_key = "<API-KEY>"

In [ ]:
from aitta_client import Model, Client, StaticAccessTokenSource
import openai
import IPython


token_source = StaticAccessTokenSource(api_key)
aitta_client = Client("https://api-staging-aitta.2.rahtiapp.fi", token_source)

# load the "allenai/OLMo-7B-0724-Instruct" model
model = Model.load("LumiOpen/Poro-34B-chat", aitta_client)
print(model.description)

# configure OpenAI client to use the Aitta OpenAI compatibility endpoints
client = openai.OpenAI(api_key=token_source.get_access_token(), base_url=model.openai_api_url)


In [ ]:
def get_response(prompt, max_completion_tokens=100):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        model=model.id,
        n=1,
        max_completion_tokens=max_completion_tokens,
        stream=False  # response streaming is currently not supported by Aitta
    )
    return response.choices[0].message.content

### 1. Sentiment analysis of feedback   <a class="anchor" id="Sentiment-analysis-of-feedback"></a>

Sentiment analysis is a natural language processing (NLP) technique used to determine the emotional tone behind text. It classifies text into categories such as **Positive, Neutral, or Negative**.

In this example, we use an LLM to process research-related feedback and automatically assign sentiment labels. 

In [ ]:
instructs = """
You are an assistant who classifies text into different categories by sentiment. 
Classify listed feedbacks into neutral, negative or positive.
Format your response as JSON with 'feedback_id' and 'category' keys.
Provide only the JSON.
"""

feedback_texts = """
1. "The research process was smooth, and I got the results I needed quickly."
2. "It took a long time to receive my dataset, but the quality was acceptable."
3. "I was extremely frustrated with the lack of communication and support."
4. "The documentation was helpful, though I had to figure out some details myself."
5. "Fantastic support and very well-organized data!"
"""

prompt = instructs + feedback_texts
# Get sentiment analysis response
sentiment_analysis = get_response(prompt)
IPython.display.Markdown(sentiment_analysis)

### 2. Text summarization <a class="anchor" id="Text-summarization"></a>

LLMs can quickly extract key insights from long research papers. 

In [ ]:
def summarize_paper(abstract):
    prompt = f"""
    You are a helpful summariser.
    Summarize the key findings and contributions in a three bullet points of the following research 
    abstract:\n{abstract}. Keep your answer clear and concise."""
    return get_response(prompt)

#https://arxiv.org/pdf/1706.03762: Attention Is All You Need
abstract = """
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-
to-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,
our model establishes a new single-model state-of-the-art BLEU score of 41.8 after
training for 3.5 days on eight GPUs, a small fraction of the training costs of the
best models from the literature. We show that the Transformer generalizes well to
other tasks by applying it successfully to English constituency parsing both with
large and limited training data
"""

summary = summarize_paper(abstract)
IPython.display.Markdown(summary)

### 3. Retrieving relevant research papers <a class="anchor" id="Retrieving-relevant-research-papers"></a>

Researchers often need to find relevant papers based on specific topics, keywords, or publication years. Manually searching through large datasets can be time-consuming, but LLMs can assist by understanding queries and retrieving relevant research papers.  

In this example, we use an LLM to search through a small mock dataset of research papers. The model identifies papers related to a given query, such as topics on **climate change, biodiversity or agriculture**, and filters results accordingly.

After that we try to create a mock dataset using LLM.

*Mock data means random data that follows a predefined column format*.

In [ ]:
# Mock dataset of research papers, created by AI
research_papers = [
    {"title": "Impact of Climate Change on Marine Life", "year": 2023, "keywords": ["climate change", "marine life"]},
    {"title": "Biodiversity and Ecosystem Services in Urban Areas", "year": 2022, "keywords": ["biodiversity", "urban areas"]},
    {"title": "Climate Change Effects on Agriculture", "year": 2021, "keywords": ["climate change", "agriculture"]},
    {"title": "Biodiversity Loss in Coral Reef Ecosystems", "year": 2023, "keywords": ["biodiversity", "coral reefs", "climate change"]},
    {"title": "Sustainable Energy Solutions and Climate Mitigation", "year": 2022, "keywords": ["renewable energy", "climate change", "sustainability"]},
    {"title": "Conservation Strategies for Endangered Species", "year": 2023, "keywords": ["conservation", "biodiversity", "endangered species"]}
]

def query_research_papers(query, dataset):
    prompt = f"Given the following research papers, find those relevant to the query: {query}\nDataset: {dataset}"
    return get_response(prompt)

query = "Find all papers published on climate change in year 2023. List the papers by titles without description."
response = query_research_papers(query, research_papers)
#print(f"Query Response: {response}")
IPython.display.Markdown(response)


In [ ]:
prompt = """
Do not respond as an assistant. Ensure strict JSON compliance.
Generate a mock dataset of research papers about climate change from the years 2015 to 2025.

Return the output **strictly** as a valid JSON array, without any additional text, explanations, or formatting. The JSON array should contain multiple objects, each with the following keys:
- "title" (string): The research paper title.
- "year" (integer): A publication year between 2015 and 2025.
- "keywords" (array of strings): Relevant keywords for the paper.

### Output Format:
```json
[
  {"title": "Impact of Climate Change on Marine Life", "year": 2018, "keywords": ["climate change", "marine life"]},
  {"title": "AI and Climate Modeling", "year": 2023, "keywords": ["AI", "climate modeling"]}
]
DO NOT include any explanations, introductions, or additional text. Only return valid JSON with 5 objects. 
"""
create_mock_dataset = get_response(prompt, 300)
IPython.display.Markdown(create_mock_dataset)


### 4. Named entity recognition (NER) for extracting key information <a class="anchor" id="NER"></a>

Named Entity Recognition (NER) is a natural language processing (NLP) task that involves identifying and classifying key entities in a given text. These entities can include people, organizations, locations, dates, and other specific terms that essential for understanding the content. NER is widely used in information extraction systems, question answering, and text summarization.

In [ ]:
#https://okm.fi/en/-/new-supercomputer-to-be-located-in-kajaani-finland-gains-stronger-role-in-ai-research
text = """
The European High Performance Computing Joint Undertaking (EuroHPC) has selected the sites 
that will host the new European AI Factories. The AI factory consists of a supercomputer that
provides world-class computing power as well as completely new data sources together with 
a service centre and talent pool.

Of the seven funded projects, one belongs to the LUMI consortium, which is led by Finland. 
The LUMI AI Factory will be hosted by CSC - IT Center for Science and located in Kajaani.
"""

prompt = f"""
Identify and list the key named entities (people, organizations, places) in the following text:\n{text}. 
Prove key entities in concise way.
"""

entities = get_response(prompt, 200)
#print(f"Named Entities: {entities}")
IPython.display.Markdown(entities)

### 5. Extracting key topics from research papers <a class="anchor" id="Extracting-key-topics"></a>

In this section, we use an LLM to automatically extract the most important keywords from a research paper. Keywords are essential for identifying the core topics and concepts of the text, which is valuable for summarizing and categorizing research. The function `extract_keywords()` takes the input text, constructs a prompt to instruct the model to identify key terms, and returns a list of significant keywords. This process helps quickly understand the main focus of the research without needing to read the entire paper.


In [ ]:
def extract_keywords(text):
    prompt = f"Extract 5 most important keywords from the following abstract:\n{text}"
    return get_response(prompt)

#https://www.researchgate.net/publication/361665646_Evaluating_GPU_Programming_Models_for_the_LUMI_Supercomputer
research_abstract = """
It is common in the HPC community that the achieved performance with just CPUs is limited 
for many computational cases. The EuroHPC pre-exascale and the coming exascale systems are 
mainly focused on accelerators, and some of the largest upcoming supercomputers such as LUMI 
and Frontier will be powered by AMD Instinct™ accelerators. However, these new systems create
many challenges for developers who are not familiar with the new ecosystem or with the 
required programming models that can be used to program for heterogeneous architectures. 
In this paper, we present some of the more well-known programming models to program for 
current and future GPU systems. We then measure the performance of each approach using a 
benchmark and a mini-app, test with various compilers, and tune the codes where necessary. 
Finally, we compare the performance, where possible, between the NVIDIA Volta (V100), 
Ampere (A100) GPUs, and the AMD MI100 GPU.
"""

keywords = extract_keywords(research_abstract)

IPython.display.Markdown(keywords)

### 6. Refining academic writing <a class="anchor" id="Refining-academic-writing"></a>

In this section, we use an LLM to improve the clarity, grammar, and readability of a rough draft of an academic research paragraph. The function `improve_writing()` sends a prompt to the model, instructing it to enhance the provided text while preserving scientific accuracy. 


In [ ]:
# Draft of a research paragraph
rough_paragraph = """
The usage of AI in biomedical sciences is growing rapidly. 
However, due to the complexity of the models and the necessity of big data, 
it sometimes leads to issues in explainability and reproducibility of results. 
As a consequence, scientists are investigating methods to make AI models more transparent, 
which in turn could improve trust in AI-based medical diagnoses.
"""

# Prompt for improving clarity, grammar, and readability
def improve_writing(text):
    prompt = f"""
    You excell at academic writing. Improve the clarity, grammar, and readability of the following 
    academic text while maintaining its scientific accuracy:\n{text}
    Response strictly the corrected text.
    """
    return get_response(prompt)

# Generate the improved version
improved_text = improve_writing(rough_paragraph)
IPython.display.Markdown(improved_text)


### 7. Translation from Finnish to English <a class="anchor" id="Translation"></a>

In this section, we use an LLM to translate text from Finnish to English. The function `translate()` sends a prompt to the model, instructing it to translate the provided text into English.

In [ ]:
def translate(text):
    prompt = f"Translate the following text from Finnish to English:\n{text}"
    return get_response(prompt)

finnish_text = """
Tekoälyn soveltaminen terveydenhuollossa tarjoaa merkittäviä mahdollisuuksia, mutta samalla se tuo mukanaan myös haasteita, 
kuten mallien läpinäkyvyyden puutteen ja tulosten toistettavuuden ongelmat. Näiden haasteiden ratkaisemiseksi tutkijat 
kehittävät keinoja, joilla tekoälyn mallit olisivat ymmärrettävämpiä ja luotettavampia.
"""

translated_text = translate(finnish_text)

IPython.display.Markdown(translated_text)